In [ ]:
!pip install lime

In [ ]:
import tensorflow as tf
from tensorflow import keras
import sklearn
import sklearn.datasets
import sklearn.ensemble
import numpy as np
import lime
import lime.lime_tabular

In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Dense, Conv1D, Input, Reshape, Permute, Add, Flatten, BatchNormalization, Activation, MaxPooling1D, Concatenate,Dropout, AveragePooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D, UpSampling1D
from keras.regularizers import l2, l1, l1_l2


# The Speck cipher and data generation algorithms #
Taken from https://github.com/agohr/deep_speck/blob/master/speck.py

In [ ]:
import numpy as np
from os import urandom

def WORD_SIZE():
    return(16);

def ALPHA():
    return(7);

def BETA():
    return(2);

MASK_VAL = 2 ** WORD_SIZE() - 1;

def shuffle_together(l):
    state = np.random.get_state();
    for x in l:
        np.random.set_state(state);
        np.random.shuffle(x);

def rol(x,k):
    return(((x << k) & MASK_VAL) | (x >> (WORD_SIZE() - k)));

def ror(x,k):
    return((x >> k) | ((x << (WORD_SIZE() - k)) & MASK_VAL));

def enc_one_round(p, k):
    c0, c1 = p[0], p[1];
    c0 = ror(c0, ALPHA());
    c0 = (c0 + c1) & MASK_VAL;
    c0 = c0 ^ k;
    c1 = rol(c1, BETA());
    c1 = c1 ^ c0;
    return(c0,c1);

def dec_one_round(c,k):
    c0, c1 = c[0], c[1];
    c1 = c1 ^ c0;
    c1 = ror(c1, BETA());
    c0 = c0 ^ k;
    c0 = (c0 - c1) & MASK_VAL;
    c0 = rol(c0, ALPHA());
    return(c0, c1);

def expand_key(k, t):
    ks = [0 for i in range(t)];
    ks[0] = k[len(k)-1];
    l = list(reversed(k[:len(k)-1]));
    for i in range(t-1):
        l[i%3], ks[i+1] = enc_one_round((l[i%3], ks[i]), i);
    return(ks);

def encrypt(p, ks):
    x, y = p[0], p[1];
    for k in ks:
        x,y = enc_one_round((x,y), k);
    return(x, y);

def decrypt(c, ks):
    x, y = c[0], c[1];
    for k in reversed(ks):
        x, y = dec_one_round((x,y), k);
    return(x,y);

def check_testvector():
  key = (0x1918,0x1110,0x0908,0x0100)
  pt = (0x6574, 0x694c)
  ks = expand_key(key, 22)
  ct = encrypt(pt, ks)
  if (ct == (0xa868, 0x42f2)):
    print("Testvector verified.")
    return(True);
  else:
    print("Testvector not verified.")
    return(False);

#convert_to_binary takes as input an array of ciphertext pairs
#where the first row of the array contains the lefthand side of the ciphertexts,
#the second row contains the righthand side of the ciphertexts,
#the third row contains the lefthand side of the second ciphertexts,
#and so on
#it returns an array of bit vectors containing the same data
def convert_to_binary(arr):
  X = np.zeros((4 * WORD_SIZE(),len(arr[0])),dtype=np.uint8);
  for i in range(4 * WORD_SIZE()):
    index = i // WORD_SIZE();
    offset = WORD_SIZE() - (i % WORD_SIZE()) - 1;
    X[i] = (arr[index] >> offset) & 1;
  X = X.transpose();
  return(X);

#takes a text file that contains encrypted block0, block1, true diff prob, real or random
#data samples are line separated, the above items whitespace-separated
#returns train data, ground truth, optimal ddt prediction
def readcsv(datei):
    data = np.genfromtxt(datei, delimiter=' ', converters={x: lambda s: int(s,16) for x in range(2)});
    X0 = [data[i][0] for i in range(len(data))];
    X1 = [data[i][1] for i in range(len(data))];
    Y = [data[i][3] for i in range(len(data))];
    Z = [data[i][2] for i in range(len(data))];
    ct0a = [X0[i] >> 16 for i in range(len(data))];
    ct1a = [X0[i] & MASK_VAL for i in range(len(data))];
    ct0b = [X1[i] >> 16 for i in range(len(data))];
    ct1b = [X1[i] & MASK_VAL for i in range(len(data))];
    ct0a = np.array(ct0a, dtype=np.uint16); ct1a = np.array(ct1a,dtype=np.uint16);
    ct0b = np.array(ct0b, dtype=np.uint16); ct1b = np.array(ct1b, dtype=np.uint16);
    
    #X = [[X0[i] >> 16, X0[i] & 0xffff, X1[i] >> 16, X1[i] & 0xffff] for i in range(len(data))];
    X = convert_to_binary([ct0a, ct1a, ct0b, ct1b]); 
    Y = np.array(Y, dtype=np.uint8); Z = np.array(Z);
    return(X,Y,Z);

#baseline training data generator
def make_train_data(n, nr, diff=(0x0040,0)):
  Y = np.frombuffer(urandom(n), dtype=np.uint8); Y = Y & 1;
  keys = np.frombuffer(urandom(8*n),dtype=np.uint16).reshape(4,-1);
  plain0l = np.frombuffer(urandom(2*n),dtype=np.uint16);
  plain0r = np.frombuffer(urandom(2*n),dtype=np.uint16);
  plain1l = plain0l ^ diff[0]; plain1r = plain0r ^ diff[1];
  num_rand_samples = np.sum(Y==0);
  plain1l[Y==0] = np.frombuffer(urandom(2*num_rand_samples),dtype=np.uint16);
  plain1r[Y==0] = np.frombuffer(urandom(2*num_rand_samples),dtype=np.uint16);
  ks = expand_key(keys, nr);
  ctdata0l, ctdata0r = encrypt((plain0l, plain0r), ks);
  ctdata1l, ctdata1r = encrypt((plain1l, plain1r), ks);
  X = convert_to_binary([ctdata0l, ctdata0r, ctdata1l, ctdata1r]);
  return(X,Y);

#real differences data generator
def real_differences_data(n, nr, diff=(0x0040,0)):
  #generate labels
  Y = np.frombuffer(urandom(n), dtype=np.uint8); Y = Y & 1;
  #generate keys
  keys = np.frombuffer(urandom(8*n),dtype=np.uint16).reshape(4,-1);
  #generate plaintexts
  plain0l = np.frombuffer(urandom(2*n),dtype=np.uint16);
  plain0r = np.frombuffer(urandom(2*n),dtype=np.uint16);
  #apply input difference
  plain1l = plain0l ^ diff[0]; plain1r = plain0r ^ diff[1];
  num_rand_samples = np.sum(Y==0);
  #expand keys and encrypt
  ks = expand_key(keys, nr);
  ctdata0l, ctdata0r = encrypt((plain0l, plain0r), ks);
  ctdata1l, ctdata1r = encrypt((plain1l, plain1r), ks);
  #generate blinding values
  k0 = np.frombuffer(urandom(2*num_rand_samples),dtype=np.uint16);
  k1 = np.frombuffer(urandom(2*num_rand_samples),dtype=np.uint16);
  #apply blinding to the samples labelled as random
  ctdata0l[Y==0] = ctdata0l[Y==0] ^ k0; ctdata0r[Y==0] = ctdata0r[Y==0] ^ k1;
  ctdata1l[Y==0] = ctdata1l[Y==0] ^ k0; ctdata1r[Y==0] = ctdata1r[Y==0] ^ k1;
  #convert to input data for neural networks
  X = convert_to_binary([ctdata0l, ctdata0r, ctdata1l, ctdata1r]);
  return(X,Y);


# The depth-1/10 distinguisher implementation - Pruned version#
Taken from https://github.com/agohr/deep_speck/blob/master/train_nets.py and slightly adapted for running multiple trials. 


In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Dense, Conv1D,Conv2D, Input, Reshape, Permute, Add, Flatten, BatchNormalization, Activation, MaxPooling1D, Concatenate,Dropout, AveragePooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D, UpSampling1D
from keras.regularizers import l2, l1, l1_l2


def cyclic_lr(num_epochs, high_lr, low_lr):
  res = lambda i: low_lr + ((num_epochs-1) - i % num_epochs)/(num_epochs-1) * (high_lr - low_lr);
  return(res);

def make_checkpoint_best_worst(datei):
  res = ModelCheckpoint(datei, monitor='val_acc', save_best_only = True, save_weights_only=True );
  return(res);

def make_checkpoint_all(datei):
  res = ModelCheckpoint(datei, monitor='val_acc', save_best_only = False, save_weights_only=True);
  return(res);


wdir_curent_bw = "./current_bw/";
wdir_curent_bw_all= "./current_bw_all/";

bs = 5000;

def make_resnet( num_blocks=2, num_filters=32, num_outputs=1, d1=64, d2=64, word_size=16, ks=3,depth=5, reg_param=0.0001, final_activation='sigmoid'):
  #Input and preprocessing layers
  #Input and preprocessing layers

  #input is 10^7 samples of 64 bits each
  inp = Input(shape=(64,));
  x = Reshape((4, 16))(inp);
  x = Permute((2,1))(x);
  
  conv0 = Conv1D(25, kernel_size=1, padding='same')(x);
  conv0 = BatchNormalization()(conv0);
  conv0 = Activation('relu')(conv0);

  conv1 = Conv1D(11, kernel_size=ks, padding='same')(conv0);
  conv1 = BatchNormalization()(conv1);
  conv1 = Activation('relu')(conv1);

  conv2 = Conv1D(8, kernel_size=ks, padding='same')(conv1);
  conv2 = BatchNormalization()(conv2);
  conv2 = Activation('relu')(conv2);

  flat = Flatten()(conv2);

  dense1 = Dense(18)(flat);
  dense1 = BatchNormalization()(dense1);
  dense1 = Activation('relu')(dense1);
  dense2 = Dense(28)(dense1);
  dense2 = BatchNormalization()(dense2);
  dense2 = Activation('relu')(dense2);
  out = Dense(1, activation='sigmoid')(dense2);
  model = Model(inputs=inp, outputs=out);

  return model

def model_builder(depth):

  model = make_resnet(depth=depth);
  model.compile(
          optimizer='adam',
          loss='binary_crossentropy',
          metrics=['acc']);
  return model;
  



def train_speck_distinguisher(model, num_epochs, num_rounds, X_train, Y_train, X_eval, Y_eval):
    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, restore_best_weights= True);
    lr = LearningRateScheduler(cyclic_lr(10,0.002, 0.0001));
    history = model.fit(X_train, Y_train, batch_size= bs, epochs=num_epochs,validation_data=(X_eval, Y_eval), callbacks=[lr, stop_early])
    
    return(model);


Names for the 64 bits

In [ ]:
def generate_names():
  names =[];
  for i in range(64):
    names.append("B"+str(i));

  return names;

In [ ]:
feature_names = generate_names()

# Train distinguisher #

In [ ]:
num_rounds = 5; # 6, 7, 8

X_train, Y_train = make_train_data(10**7, num_rounds);
X_eval, Y_eval = make_train_data(10**6, num_rounds);

initial_model = model_builder(1);
trained_model= train_speck_distinguisher(initial_model, 30, num_rounds, X_train, Y_train, X_eval, Y_eval);

# Find samples with best/worst prediction #

In [ ]:
def find_best_preds(trained_model, num_preds, num_rounds):
    X_eval, Y_eval = make_train_data(10**6, num_rounds);
    predictions= trained_model.predict(X_eval, batch_size=5000)
    stored_0 =[];
    stored_1 =[];
    
    for i in range(10**6):
        diff = abs(Y_eval[i] - predictions[i]);
        if Y_eval[i] ==1 and (len(stored_1)==0 or diff <= stored_1[len(stored_1)-1][1]):
            stored_1.append((i, diff, Y_eval[i], predictions[i], X_eval[i]));
        if Y_eval[i] ==0 and (len(stored_0)==0 or diff <= stored_0[len(stored_0)-1][1]):
            stored_0.append((i, diff, Y_eval[i], predictions[i], X_eval[i]));
    return stored_1[-num_preds:], stored_0[-num_preds:];
        

In [ ]:
def find_worst_preds(trained_model, num_preds, num_rounds):
    X_eval, Y_eval = make_train_data(10**6, num_rounds);
    predictions= trained_model.predict(X_eval, batch_size=5000)
    stored_0 =[];
    stored_1 =[];
    
    for i in range(10**6):
        diff = abs(Y_eval[i] - predictions[i]);
        if Y_eval[i] ==1 and (len(stored_1)==0 or diff >= stored_1[len(stored_1)-1][1]):
            stored_1.append((i, diff, Y_eval[i], predictions[i], X_eval[i]));
        if Y_eval[i] ==0 and (len(stored_0)==0 or diff >= stored_0[len(stored_0)-1][1]):
            stored_0.append((i, diff, Y_eval[i], predictions[i], X_eval[i]));
    return stored_1[-num_preds:], stored_0[-num_preds:];

In [ ]:
s1, s2 = find_best_preds(trained_model, 5, 5);

# Instantiate the explainer #

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(X_train, class_names= ['Fixed difference'], feature_names=feature_names,discretize_continuous=False);

Explanations for class 1 - fixed difference

In [ ]:
exp = explainer.explain_instance(s1[0][4], trained_model.predict, num_features=64, num_samples=10**7, labels=[0])
exp.show_in_notebook(show_table=True, show_all=True)
fig = exp.as_pyplot_figure(label=0);
fig.set_size_inches(10, 15)
#fig

Explanations for class 0 - random difference

In [ ]:
exp = explainer.explain_instance(s2[0][4], trained_model.predict, num_features=64, num_samples=10**7, labels=[0])
exp.show_in_notebook(show_table=True, show_all=True)
fig = exp.as_pyplot_figure(label=0);
fig.set_size_inches(10, 15)

# Submodular Explainer #

In [ ]:
num_rounds = 5; # 6, 7, 8

X_train, Y_train = make_train_data(10**7, num_rounds);
X_eval, Y_eval = make_train_data(10**6, num_rounds);

initial_model = model_builder(1);
trained_model= train_speck_distinguisher(initial_model, 30, num_rounds, X_train, Y_train, X_eval, Y_eval);

In [ ]:
import warnings
from lime import submodular_pick
sp_obj = submodular_pick.SubmodularPick(explainer, X_train, trained_model.predict, method='sample', num_features=64, num_exps_desired=5,  sample_size =5000) 
[exp.as_pyplot_figure(label=exp.available_labels()[0]).set_size_inches(10, 15) for exp in sp_obj.sp_explanations];

In [ ]:
[exp.show_in_notebook(show_table=True, show_all=True) for exp in sp_obj.sp_explanations];